In [17]:
import ee

In [18]:
ee.Authenticate()

True

In [19]:
ee.Initialize()

In [20]:
import ee
import geemap
import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import gc
from tqdm import tqdm

#Map = geemap.Map()
city_ls = ["Nairobi"]
#city_ls = ["Abidjan","Abuja","Accra","Bamako","Conakry","Dakar","Lagos","Libreville","Lome","NDjamena","Ouagadougou"]

p = "/home/ez2422/"

**Nairobi (-1.2913, 36.8227): Kenyatta University-City Centre Campus, Nairobi, Kenya**

In [21]:
'''
lat_d = {
    "Abidjan":5.3350,
    "Abuja":9.0405,
    "Accra":5.5795,
    "Bamako":12.6298,
    "Conakry":9.5950,
    "Dakar":14.7452,
    "Lagos":6.4405,
    "Libreville":0.4162,
    "Lome":6.1855,
    "NDjamena":12.1020,
    "Ouagadougou":12.3049
    "Nairobi":-1.2913
}
lon_d = {
    "Abidjan":-3.9760,
    "Abuja":7.4765,
    "Accra":-0.1706,
    "Bamako":-8.0189,
    "Conakry":-13.6366,
    "Dakar":-17.5262,
    "Lagos":3.4067,
    "Libreville":9.4673,
    "Lome":1.2142,
    "NDjamena":15.0479,
    "Ouagadougou":-1.4972
    "Nairobi":36.8227
}
'''
lat_d = {
    "Nairobi":-1.2913
}
lon_d = {
    "Nairobi":36.8227
}

product_d = {
    "CO":"COPERNICUS/S5P/NRTI/L3_CO",
    "HCHO":"COPERNICUS/S5P/NRTI/L3_HCHO",
    "O3":"COPERNICUS/S5P/NRTI/L3_O3",
    "SO2":"COPERNICUS/S5P/NRTI/L3_SO2",
    "NO2":"COPERNICUS/S5P/NRTI/L3_NO2",
    "UVAI":"COPERNICUS/S5P/NRTI/L3_AER_AI",
    "UVLH":"COPERNICUS/S5P/NRTI/L3_AER_LH"
}

var_d = {
    "CO":["CO_column_number_density"],
    "HCHO":["tropospheric_HCHO_column_number_density"],
    "O3":["O3_column_number_density"],
    "SO2":["SO2_column_number_density"],
    "NO2":["tropospheric_NO2_column_number_density"],
    "UVAI":["absorbing_aerosol_index"],
    "UVLH":["aerosol_height"]
}

## get remote sensing data

In [23]:
# get a time series: 
# https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    #df = df[['time','datetime','longitude','latitude',  *list_of_bands]]
    df = df[['time','datetime',  *list_of_bands]]

    return df

def get_df(s, loc, point_buffer, product_d, var_d): 
    collection = ee.ImageCollection(product_d[s]) \
        .filterBounds(loc) \
        .select(var_d[s]) \
        .filterDate('2021-01-01', '2024-11-30')

    lst = collection.getRegion(loc,point_buffer).getInfo()

    df = ee_array_to_df(lst,var_d[s])
    df = df.set_index("datetime")[var_d[s]].resample("D").mean()
    df = df.reset_index()
    df["time"] = df["datetime"].dt.date
    return df[["time"]+var_d[s]]

def merge_df(s_ls, loc, point_buffer, product_d, var_d):
    for s in s_ls:
        if s == s_ls[0]:
            df = get_df(s, loc=loc, point_buffer=point_buffer, product_d=product_d, var_d=var_d)
        else:
            df = pd.merge(df, get_df(s, loc=loc, point_buffer=point_buffer, product_d=product_d, var_d=var_d), 
                          on="time", how="outer")
    return df

In [24]:
import pandas as pd
from tqdm import tqdm
import ee

s_ls = ["NO2"]  # Only NO2
start_date = "2021-01-01"

for city in tqdm(city_ls):
    loc = ee.Geometry.Point(lon_d[city], lat_d[city])
    point_buffer = 1500.0
    df = merge_df(s_ls, loc, point_buffer, product_d, var_d)

    # Convert time to datetime and filter date range
    df["time"] = pd.to_datetime(df["time"])
    df = df[df["time"] >= start_date]  # From 2021-01-01 to present
    df = df.sort_values("time").reset_index(drop=True)

    df.to_csv(p + city + ".csv", index=False)

100%|██████████| 1/1 [00:06<00:00,  6.06s/it]


In [25]:
for city in tqdm(city_ls):
    print(city)
    df = pd.read_csv(p+city+".csv")
    display(df.describe())
    print("\n")

  0%|          | 0/1 [00:00<?, ?it/s]

Nairobi


,tropospheric_NO2_column_number_density
count,573.000000
mean,0.000044
std,0.000020
min,-0.000006
25%,0.000030
50%,0.000041
75%,0.000053
max,0.000125


100%|██████████| 1/1 [00:00<00:00, 104.12it/s]

In [26]:
# Pickle file for further use
df.to_pickle("my_df2.pkl")